# Colouring of graphs of polytopes

**Guillermo Pineda-Villavicencio** and **Julien Ugon**

*(Deakin University, Australia)*

## Abstract


Hadwiger conjectured the following. 

**Hadwidger's conjecture:** For every $t\ge 2$, every graph with no $K_{t}$-minor is $(t-1)$-vertex-colourable. 

Every topological minor is a minor, and so the following conjecture of Hajos strengthens Hadwiger's.  

**Hajos' conjecture:** For every $t\ge 2$, every graph with no $K_{t}$-topological-minor is $(t-1)$-vertex-colourable. 

In the report we consult the available  databases for polytopes (Firsching, n.d.) and (Miyata, Moriyama, and Fukuda n.d.), and established the veracity of   Hajos', and thus of Hadwiger's,   for $d$-polytopes on $n$ vertices for the pairs $(d,n)=(4,6-9),(5,7-9),(6,8-9),(7,9)$. 

Catlin (1979) gave counterexamples to Hajos' conjecture for all $t\ge 7$. His counterexamples are obtained from line graphs of multicycles whose edges have been replaced with multiple edges. The multigraph $_k{C_{2n+1}}$ is obtained from the cycle $C_{2n+1}$ of length $2n+1$ by replacing each edge $xy$ with $k$ edges joining $x$ to $y$. The <i>line graph</i> of $_k{C_{2n+1}}$ is denoted by $L(_k{C_{2n+1}})$.

Let $G_{3,5}$ denote the graph obtained from $L(_3{C_{5}})$ by deleting two nonadjavent vertices. Catlin showed that the chromatic number of $G_{3,5}$ is 7, while its <i>subdivision number</i>, the largest integer $t$ such that $G_{3,5}$ has $K_t$ as topological minor, is 6. Hence it violates Hajos' conjecture. We show that $G_{3,5}$ is not the graph of a polytope.   

In addition, consulting the database of the polytopal simplicial 3-spheres on a small number of vertices (Firsching, n.d.), we answered affirmatively the following question of Thompson question for simple $4$-polytopes a small number of facets, namely 6-9 facets (See Kalai 2014). 

**Thompson's question:** Are graphs of simple $d$-polytopes with an even number of vertices $d$-edge-colourable? 

## 1. Introduction

We report on the results of experiments on the validity of Hajós’ and Thomson’s conjectures on graphs of polytopes.
This is a Sage notebook, and therefore, to run it you need to have Sage (Stein and Joyner 2005) installed.

The latest version of  this  file can be found  at (Ugon,  n.d.)

In order to run this program, please download the files provided at http://www-imai.is.s.u-tokyo.ac.jp/~hmiyata/oriented_matroids/ and unzip the files in the folder.
The list of files to  download are:
* polytope_n5d4.txt
* polytope_n6d4.txt
* polytope_n7d4.txt
* polytope_n8d4.txt
* polytope_n6d5.txt
* polytope_n7d5.txt
* polytope_n8d5.txt
* polytope_n9d5.zip *(this file needs to be unziped)*
* polytope_n7d6.txt
* polytope_n8d6.txt
* polytope_n9d6.txt
* polytope_n8d7.txt
* polytope_n9d7.txt

If the files are not found in the folder, they will be downloaded automatically.

These files contain all $d$-polytopes on $n$ vertices for the pairs $(d,n)=(4,6-8),(5,7-9),(6,8-9),(7,9)$.

You will also need to download some files provided at http://page.mi.fu-berlin.de/moritz/polys/inscribe/, which contain additional polytopes, namely all 4-polyopes on 9 vertices and all simplicial polytopes on 9 vertices. These files are:
* all4polytopes9vertices.txt
* polytopalsimplicialspheres9_4.txt

Again, these files will be downloaded automatically if they are not found in the folder.

Finally, if you are running this notebook, a word of advice: we have left the experiment on checking Hajos' conjecture on 4-polytopes on 9 vertices for the end of this notebook, because this experiment takes a long time. You should be able to run every other experiment before this one in a reasonable time.

### Initial functions and preliminaries

In [1]:
from urllib.request import urlretrieve
import zipfile
from pathlib import Path

In [2]:
## Download the files if necessary

polytopeParameters = [(4,6),(4,7),(4,8),(5,7),(5,8),(5,9),(6,8),(6,9),(7,9)]

stem = "http://www-imai.is.s.u-tokyo.ac.jp/~hmiyata/oriented_matroids/"

polytopeFiles = ["polytope_n%dd%d"%(n,d) for (d,n) in polytopeParameters]

for f in polytopeFiles:
    if not Path(f+".txt").is_file():
        print("Retrieving "+f+".txt")
        try:
            urlretrieve(stem+f+".txt",f+".txt")
        except:
            urlretrieve(stem+f+".zip",f+".zip")
            with zipfile.ZipFile(f+".zip", 'r') as zfile:
                zfile.extractall("./")

In [3]:
def getFacets(fname):
    """
    Get the list of facets from a file:

    Input: A file in the format as in  (Miyata, Moriyama, and Fukuda n.d.)
    Output: A list of list of vertices. Each sublist contains the vertices of a facet.
    """
    if isinstance(fname,str):
        try:
            return getFacets(open(fname,"r"))
        except:
            return []
    
    return [[Set(eval(f)) for f in l.split()] for l in fname]

In [4]:
def intersect(*args):
    """
    Compute the intersection of sets

    Input: A list of sets
    Output: The intersection, as a set
    """
    if(len(args)) == 0:
        return Set()
    return args[0].intersection(intersect(*args[1:]))

def union(*args):
    """
    Compute the union of sets

    Input: A list of sets
    Output: The union, as a set
    """
    if(len(args)) == 0:
        return Set()
    return args[0].union(union(*args[1:]))

In [5]:
def findEdges(facets):
    """
    Given a facet-vertex incidence, find the edges.

    This works as follows:
    - take all possible pairs of vertices (C(n,2));
    - For each pair, take all the facets containing both vertices
    - The  intersection of these facets is precisely these two vertices if and only if it's an edge.

    Input: A list of lists, where each sublist is the vertex list of a facet.
    Output: A list of pairs (tuples). Each tuple represents an edge.
    """
    vertices = union(*facets)
    #take all pairs of vertices:
    V = [(v,w) for v in vertices for w in vertices if v<w]
    L = []
    U = Set(vertices)
    for (v,w) in V:
        F = [f for  f in facets if v in f and w in f]
        E = intersect(*F)
        if E.cardinality() == 2:
            L.append(E)
    return [tuple(e) for e in L]

## 2. Hajós’ conjecture

Hajós’ conjecture states that:

**Hajos' conjecture:** For every $t\geq 2$, every graph with no $K_t$-topological-minor is
$(t−1)$-vertex-colourable.

### 2.1 Small graphs

We tested Hajós’ conjecture on the graph of all the polytopes provided
by (Miyata, Moriyama, and Fukuda n.d.). More precisely, we check, on the graph of thoese polytopes, whether the following statement is true:

**Hajos' conjecture (contrapositive):** For $t\geq 2$, if $\chi(G) = t$, then $G$ contains a $K_t$-topological minor. 

We applied the following algorithm:

1.  Compute the graph of the polytope;

2.  Compute the chromatic number $t$ of the graph;

3.  Verify that $K_t$ is a topological minor of the graph.



In [6]:
def satisfiesHajos(G):
    """
    Check if a graph satisfies Hajos' conjecture.

    Input: A graph
    Output: A boolean: true if the graph satisfies Hajos, false otherwise.
    """
    # Compute the Chromatic Number
    C = G.chromatic_number();
    # Find a topological minor $K_C$ in G, if possible
    m = G.topological_minor(graphs.CompleteGraph(C))
    return m != false


def findCounterExamplesHajos(L):
    """
    Find the sublist of counter-examples to Hajós' conjecture in a list of graphs.

    Input: A list of graphs
    Output: A list of graphs that do not satisfy Hajós' conjecture (can be empty)
    """
    return [G for G in L if not satisfiesHajos(G)]


def findPolytopeCounterExamplesHajos(L):
    """
    Find the sublist of polytopes whose graph fails to satisfy Hajós' conjecture, from a list of polytopes.

    Input: A list of polytopes (given by the list of facets)
    Output: A list of polytopes whose graph do not satisfy Hajós' conjecture.
    """
    return[P for P in L if not satisfiesHajos(Graph(findEdges(P)))]

In [7]:
## Read the polyopes from the list of files:
hajos = [findPolytopeCounterExamplesHajos(getFacets("polytope_n%dd%d.txt"%(n,d))) for (d,n) in polytopeParameters]

## This will return the list of polytopes whose graphs do not satisfy Hajós' conjecture in the list.
print(sum(hajos,[]))

[]


This list is empty. Therefore we found no counterexample in the polytopes that we checked. All of them have graphs satisfying Hajós' conjecture.

### 2.2 Catlin’s Counter examples

In this section we consider Catlin’s famous counter-example to Hajós’
conjecture (Catlin 1979). In this paper, Catlin gives a family of graphs
with chromatic number $n$ that contain no subdivided $K_n$ as a
subgraph. The first counter-example is constructed by removing two non-adjacent vertices $v_1$ and $v_2$ from $L_{_3C_5}$ - namely, Catlin shows that $G_{3,5} = L(_3C_5) - \{v_1,v_2\}$ contradicts Hajós' conjecture. 

Here we show that $G_{3,5}$ is not the graph of a polytope.

In [8]:
## We start by constructing Catlin's counter-example:

G35=Graph(multiedges=False);
G35.add_edges([(1,2),(1,3),(1,4),(1,5),(1,6),(1,7),(1,8),(2,3),(2,4),(2,5),(2,6),(2,7),(2,8),(3,4),(3,5),(3,4),(3,9),(3,10),(4,5),(4,9),(4,10),(5,9),(5,10),(6,7),(6,8),(6,11),(6,12),(6,13),(7,8),(7,11),(7,12),(7,13),(8,11),(8,12),(8,13),(9,10),(9,11),(9,12),(9,13),(10,11),(10,12),(10,13),(11,12),(11,13),(12,13)])

First note that $G$ is not planar, and so cannot be the graph
of a 3-polytope. Since it is not 5-connected (the vertices
$\{1,2,9,10\}$ disconnect the graph, it
cannot be the graph of a $d$-polytope for $d\geq 5$. So, we only need to study the
case $d=4$.

In [9]:
print("The connectivity of the graph is: ",G35.vertex_connectivity())
print("Is the graph planar? ",G35.is_planar())

The connectivity of the graph is:  4
Is the graph planar?  False


The graphs of the facets of any 4-polytope must be planar, 3-connnected,
and should not disconnect the entire graph.

In [10]:
## Get all the connected subgraphs of G35 of order at most 10.
LGraphs =list(G35.connected_subgraph_iterator(k=10))


def validSubgraph(H):
    """
    A filter function that  accepts only graphs that are:
    - at least 4 vertices
    - minimum degree 3
    - is planar
    - the connectivity is at least 3
    
    Input: A list of Graphs
    Output: A boolean, which is true if all the criteria above are satisfied and false otherwise.
    """
    l = G35.copy()
    l.delete_vertices(H.vertices())
    return  H.num_verts() >= 4 and  min(H.degree()) > 2  and H.is_planar() and  H.vertex_connectivity() >2 and l.is_connected()
    # and not set(H.vertices()).issubset(P)

##l2  =filter(validSubgraph,lm35.connected_subgraph_iterator(k=10))
l2  = list(filter(validSubgraph,LGraphs))

Let us consider those that
contain Vertex $3$ and either of vertices 1,2 and 9. The graphs of
these facets must be one of the following subgraphs of $G$.

We start by finding the subsets containing the edge {1,3}.

In [11]:
## Get the 3-connected, planar subgraphs containing the edge (1,3):
S13 = [l for l in l2 if (1,3,None) in l.edges() or (3,1,None) in l.edges()]
print([s.vertices() for s  in S13])

[[1, 3, 4, 5, 10], [1, 2, 3, 4], [1, 2, 3, 5], [1, 3, 4, 5], [1, 3, 4, 5, 9]]


The edge $\{1,3\}$ must belong to at least three facets amongs the candidates:
$\{1,2,3,4\}$,$\{1,2,3,5\}$,$\{1,3,4,5,9\}$,$\{1,3,4,5,10\}$,
and $\{1,3,4,5\}$. At least one of these three facets must contain the
vertices $\{3,4,5\}$. 


By a similar argument, the edge $\{2,3\}$ must
also belong to a facet containing the vertices $\{3,4,5\}$:

In [12]:
S23 = [l for l in l2 if (2,3,None) in l.edges() or (3,2,None) in l.edges()]
print([s.vertices() for s  in S23])

[[2, 3, 4, 5, 10], [1, 2, 3, 4], [1, 2, 3, 5], [2, 3, 4, 5], [2, 3, 4, 5, 9]]


 The
intersection between these two facets is either the triangle
$\{3,4,5\}$, or contains the four vertices $\{3,4,5,k\}$ for
$k\in \{9,10\}$. We can rule out the latter case, since this would
require the graph of this 2-face with 4 vertices to be complete.

We can conclude that the sets of vertices $\{1,3,4,5\}$ and
$\{2,3,4,5\}$ both form a facet of the polytope, and that the vertices $\{3,4,5\}$ form a ridge.

The edge$\{3,9\}$ must also belong to at least three facets, one of
which contains the ridge $\{3,4,5\}$:

In [13]:
S39 = [l for l in l2 if (3,9,None) in l.edges() or (9,3,None) in l.edges()]
print([s.vertices() for s  in S39])

[[3, 4, 9, 10], [3, 5, 9, 10], [1, 3, 4, 5, 9], [2, 3, 4, 5, 9], [3, 4, 5, 9]]


This cannot happen, since this
would imply that the ridge is contained in three distinct facets.

Therefore we conclude that Catlin's example is not the graph of a polytope.

## 3. Thompson’s conjecture

Abigail Thompson posed the following question (Kalai 2014):

**Thompson's question:** Graphs of simple $d$-polytopes with an even number of vertices are
$d$-colourable.

We tested these conjecture of small polytopes provided by (Miyata,
Moriyama, and Fukuda n.d.) and (Firsching, n.d.), using the following algorithm. Namely, we
selected all simplicial polytopes in this collection with an even number
of facets. We then verified that the chromatic numbers of each of these
polytopes is at most $d$.

In [14]:
def dualFacets(f):
    """
    This function returns the facets of the dual polytope, given a list of facets for the primal.

    Input: A list of lists, where each sublist is the vertex list of a facet.
    Output: A similar list, for the dual polytope.
    """
    nvertices = len(f)
    facets = [Set([i for (i,facet) in enumerate(f) if v in facet]) for v in union(*f)]
    return facets


def isSimplicial(facets,d):
    """
    Verifies that a polytope is simplicial.

    Input: A list of lists, where each sublist is the vertex list of a facet.
    Output: A boolean: true if the polytope is simplicial, false otherwise.
    """
    #return all(f.cardinality() == d for f in facets)
    return all(len(f) == d for f in facets)

In [15]:
def satisfiesThompson(G,d):
    """
    Check if a polytope satisfies Thompson's question.

    Input: A graph, and a dimension of a polytope
    Output: A boolean: true if the graph satisfies Hajos, false otherwise.
    """
    return d >= graph_coloring.edge_coloring(G,value_only=true)

def findPolytopeCounterExamplesThompson(L):
    """
    Find the sublist of polytopes whose graph fails to satisfy Hajós' conjecture, from a list of polytopes.

    Input: A list of simple polytopes (given by the list of facets)
    Output: A list of polytopes whose graph do not satisfy Hajós' conjecture.
    """
    return[P for P in L if not satisfiesThompson(Graph(findEdges(dualFacets(P))),P.dimension())]

In [16]:
## Read the polyopes from the list of files:
polytopes = [(getFacets("polytope_n%dd%d.txt"%(n,d)),d)  for (d,n) in polytopeParameters]

# Get the lists of polytopes which fail to satisfy Thompson's question:
thompson = [findPolytopeCounterExamplesThompson(F) for (F,d) in polytopes if len(F)%2 == 0 and isSimplicial(F,d)]

## This will return the list of polytopes whose graphs do not satisfy Thompson's question from list.
print(sum(thompson,[]))

[]


We check separately the case $d=4$ and $n=9$, because  it comes from a different database (Firsching, n.d.), in a different format. 

This is done below.

In [17]:
def checkThompson(lines):
    """
    Check Thompson's questions on graphs of simplicial polytopes, with even number of vertices.
    More specifically, find a list of polytopes that fail to satisfy Thompson's question.

    Input: lines containing realisation of polyopes;
    Output: a list of triplets containing the polytope, its graph, and the pair with the chromatic number and the dimension
    """
    polytopes = [Polyhedron(vertices=eval(l[l.index("["):]),base_ring=QQ) for l in lines]
    evenPolytopes = [P.polar() for P in polytopes if P.n_facets()%2 == 0]
    return [P for P in evenPolytopes if not satisfiesThompson(P.graph(),P.dimension())]

In [18]:
# First, retrieve the file:
stem = "ftp://ftp.imp.fu-berlin.de/pub/moritz/inscribe/"
fname = "polytopalsimplicialspheres9_4.txt"

if not Path(fname).is_file():
    print("Retrieving "+fname)
    urlretrieve(stem+fname,fname)

In [19]:
# Compute all the counter-examples from the file polytopalsimplicialspheres9_4.txt
with open(fname,"r") as polytopeFile:
    print(checkThompson(polytopeFile.readlines()))

[]


## References

<div id="refs" class="references hanging-indent">

<div id="ref-catlin1979hajos">

Catlin, Paul A. 1979. “Hajós’ Graph-Coloring Conjecture: Variations and
Counterexamples.” *Journal of Combinatorial Theory, Series B* 26 (2):
268–74.

</div>
<div id="ref-firsching">

M. Firsching. “Classifying, realising...”,  Accessed January 3, 2020, <http://page.mi.fu-berlin.de/moritz/polys/inscribe/>.

</div>
<div id="ref-kal14">

Kalai, Gil. 2014. “Coloring Simple Polytopes and Triangulations.” 2014.
<https://gilkalai.wordpress.com/2014/12/06/coloring-simple-polytopes-and-triangulations/>.

</div>

<div id="ref-matroids">

Miyata, Hiroyuki, Sonoko Moriyama, and Komei Fukuda. n.d.
“Classification of Oriented Matroids.” Accessed January 3, 2020.
<http://www-imai.is.s.u-tokyo.ac.jp/~hmiyata/oriented_matroids/>.

</div>

<div id="ref-SteinJoyner2005">

Stein, William, and David Joyner. 2005. “SAGE: System for Algebra and
Geometry Experimentation.” *ACM SIGSAM Bulletin* 39 (2): 61–64.
<http://www.sagemath.org/files/sage_stein2005.pdf>.

</div>

<div id="ref-repo">

Ugon, Julien. n.d. “Github Repository.”
<https://github.com/ugonj/Graphs-of-Polytopes>.

</div>



## Appendix

Here we put code that we do not advise you  to  run, as it takes a very long time.

Namely, this code verifies Hajós' conjecture on 4-dimensional polytopes with 9 vertices. It does so in several batches, so that the program can be interrupted and started again.

In [20]:
# First, retrieve the file:
stem = "ftp://ftp.imp.fu-berlin.de/pub/moritz/classification/"
fname = "all4polytopes9vertices.txt"

if not Path(fname).is_file():
    print("Retrieving "+fname)
    urlretrieve(stem+fname,fname)

In [21]:
"""
Check Hajos' conjecture on graphs of polytopes. More specifically:
- Compute  the chromatic number t of the graph;
- Check that the graph has K_t as a topological minor.

Returns a list of all counter-examples to Hajós' conjecture
"""
def checkHajos(lines):
    polys = []
    for l in lines:
       P = Polyhedron(vertices=eval(l[l.index("["):]),base_ring=QQ)
       G = P.graph()
       C = G.chromatic_number();
       #m = findLargestCompleteTopologicalMinor(G,6)
       m = G.topological_minor(graphs.CompleteGraph(C))
       #t = findLargestCompleteMinor(G,10)
       if not m :
           polys.append((P,G,(C,m)))
    return polys

**Do not run the cell below, unless you are ready to wait!**

The experiment has been broken down, so that it can be restarted from when it stopped if necessary. We process 10000 polytopes at each step, by default.

In [ ]:
with open(fname,"r") as polyFile:
    lines = polyFile.readlines()

# Do 10000 lines at a time:
step = 10000
start = 90000
for nLines in range(40000,len(lines),step):
    print(nLines)
    print(checkHajos(lines[nLines:nLines+step]))